# Strands Agent에서 OpenAI 모델 사용하기

## 개요

Strands Agents는 몇 줄의 코드만으로 AI 에이전트를 구축하고 실행하는 모델 중심 접근 방식을 취하는 SDK입니다. Strands는 어디서나 호스팅되는 여러 제공자와 모델을 지원합니다.

[LiteLLM](https://docs.litellm.ai/docs/)은 Amazon, Anthropic, OpenAI 등의 모델과 단일 API를 통해 상호작용할 수 있게 해주는 다양한 LLM 제공자를 위한 통합 인터페이스입니다. Strands Agent SDK는 LiteLLM 제공자를 구현하여 LiteLLM이 지원하는 모든 모델에 대해 에이전트를 실행할 수 있게 합니다.

이 예제에서는 Microsoft Azure에서 호스팅되는 `gpt-4.1-mini` 모델을 Strands Agent의 기본 모델로 사용하는 방법을 보여드리겠습니다. 날씨와 시간 가져오기 도구가 있는 간단한 에이전트 사용 사례를 사용하겠습니다.


## 에이전트 세부사항
<div style="float: left; margin-right: 20px;">
    
|기능                |설명                                               |
|--------------------|---------------------------------------------------|
|사용된 기능          |LiteLLM 모델                                       |
|에이전트 구조        |단일 에이전트 아키텍처                              |

</div>

## 아키텍처

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 주요 기능
* **LiteLLM 모델**: LiteLLM을 통해 제공되는 모델 사용

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* Azure 계정
* gpt-4.1-mini 액세스 권한

이제 Strands Agent에 필요한 패키지들을 설치하겠습니다

In [ ]:
# 사전 요구사항 설치
!pip install -r requirements.txt

### 의존성 패키지 가져오기

이제 의존성 패키지들을 가져오겠습니다

In [ ]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel

### Azure 키 설정

이제 Azure API 키를 설정하겠습니다

In [ ]:
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

### 사용자 정의 도구 설정

이제 에이전트를 테스트하기 위한 두 개의 더미 도구를 설정하겠습니다

In [ ]:
@tool
def current_time(timezone: str = "UTC") -> str:
    """현재 시간을 가져옵니다.
    
    Args:
        timezone: 시간대 (기본값: UTC)
    
    Returns:
        현재 시간 (ISO 형식)
    """
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    """현재 날씨를 가져옵니다.
    
    Args:
        city: 도시 이름
    
    Returns:
        날씨 정보
    """
    # 더미 구현입니다. 실제 날씨 API 호출로 교체하세요.
    return "맑음"

### 에이전트 기본 LLM 모델 정의

다음으로 LiteLLM을 사용하여 에이전트의 기본 모델을 정의하겠습니다. `gpt-4.1-mini`로 설정하겠습니다

In [ ]:
model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)

### 에이전트 정의

이제 필요한 모든 정보가 준비되었으므로 에이전트를 정의하겠습니다

In [ ]:
system_prompt = "당신은 시간과 날씨를 알려줄 수 있는 간단한 에이전트입니다"
agent = Agent(
    model=litellm_model,
    system_prompt=system_prompt,
    tools=[current_time, current_weather],
)

### 에이전트 테스트

이제 에이전트를 호출하여 테스트해보겠습니다

In [ ]:
results = agent("시애틀의 현재 시간은 몇 시인가요? 그리고 날씨는 어떤가요?")

#### 에이전트 결과 분석

좋습니다! 처음으로 에이전트를 호출했습니다! 이제 결과 객체를 살펴보겠습니다. 먼저 에이전트 객체에서 에이전트가 교환한 메시지를 볼 수 있습니다

In [ ]:
agent.messages

다음으로 결과 `metrics`를 분석하여 마지막 쿼리에 대한 에이전트 사용량을 살펴볼 수 있습니다

In [ ]:
results.metrics

### 축하합니다!

이 노트북에서 날씨 에이전트에 대한 답변을 제공하는 OpenAI와 함께 LiteLLM을 사용하는 방법을 배웠습니다.